# ComeBackHome with provide current posistion

In [2]:
from numba import jit, njit
import random
import numpy as np

In [3]:
@njit 
def getAgentSize():
    return 10
@njit
def getStateSize():
    return 10
@njit   
def getActionSize():
    return 4

In [16]:
@njit 
def getReward(p_state):
    if p_state[0] == p_state[1] and p_state[2] == p_state[3] and p_state[8] <= p_state[9]:
        return 1
    elif p_state[8] == p_state[9] and p_state[2] != p_state[3] and p_state[0] != p_state[1]:
        return 0
    else:
        return -1
@njit 
def checkEnded(e_state):
    size= 10
    if  e_state[size*9*size+0*2] == e_state[size*9*size+4] and e_state[size*9*size+0*2+1] == e_state[size*9*size+5] and e_state[size*9*size+6] <= e_state[size*9*size+7] :
        return 1
    return -1
    

In [93]:
# Tìm turn tối ưu 
@njit
def findTurn(p_state):
    
import sys
 
# User defined Pair class
class Pair:
    def __init__(self, x, y):
        self.first = x
        self.second = y
 
# Check if it is possible to go to (x, y) from the current
# position. The function returns false if the cell has
# value 0 or already visited
def isSafe(mat, visited, x, y):
    return (x >= 0 and x < len(mat) and y >= 0 and y < len(mat[0]) and mat[x][y] == 1 and (not visited[x][y]))
 
def findShortestPath(mat, visited, i, j, x, y, min_dist, dist):
    if (i == x and j == y):
        min_dist = min(dist, min_dist)
        return min_dist
 
    # set (i, j) cell as visited
    visited[i][j] = True
     
    # go to the bottom cell
    if (isSafe(mat, visited, i + 1, j)):
        min_dist = findShortestPath(
            mat, visited, i + 1, j, x, y, min_dist, dist + 1)
 
    # go to the right cell
    if (isSafe(mat, visited, i, j + 1)):
        min_dist = findShortestPath(
            mat, visited, i, j + 1, x, y, min_dist, dist + 1)
 
    # go to the top cell
    if (isSafe(mat, visited, i - 1, j)):
        min_dist = findShortestPath(
            mat, visited, i - 1, j, x, y, min_dist, dist + 1)
 
    # go to the left cell
    if (isSafe(mat, visited, i, j - 1)):
        min_dist = findShortestPath(
            mat, visited, i, j - 1, x, y, min_dist, dist + 1)
 
    # backtrack: remove (i, j) from the visited matrix
    visited[i][j] = False
    return min_dist
 
# Wrapper over findShortestPath() function
def findShortestPathLength(mat, src, dest):
    if (len(mat) == 0 or mat[src.first][src.second] == 0
            or mat[dest.first][dest.second] == 0):
        return -1
 
    row = len(mat)
    col = len(mat[0])
 
    # construct an `M × N` matrix to keep track of visited
    # cells
    visited = []
    for i in range(row):
        visited.append([None for _ in range(col)])
 
    dist = sys.maxsize
    dist = findShortestPath(mat, visited, src.first,
                            src.second, dest.first, dest.second, dist, 0)
 
    if (dist != sys.maxsize):
        return dist
    return -1

src = Pair(1, 1)
dest = Pair(7, 7)
mat = [[0, 0 ,0, 0 ,0, 0 ,0, 0 ,0],
        [0 ,1 ,1 ,1 ,1 ,1 ,1 ,1 ,0],
        [0 ,1 ,0 ,0 ,1 ,0 ,0 ,1 ,0],
        [0 ,1 ,0 ,0 ,1 ,0 ,0 ,1 ,0],
        [0 ,1 ,0 ,0 ,1 ,0 ,0 ,1 ,0],
        [0 ,1 ,0 ,0 ,0 ,0 ,0 ,1 ,0],
        [0 ,1 ,0 ,0 ,0 ,0 ,0 ,1 ,0],
        [0 ,1 ,0 ,0 ,1 ,1 ,1 ,1 ,0],
        [0 ,0 ,0 ,0, 0 ,0, 0 ,0, 0]]
dist = findShortestPathLength(mat, src, dest)
if (dist != -1):
    print("Shortest Path is", dist)



Shortest Path is 12


In [19]:
@njit   
def initEnv():
    n = 1
    p = 0.5
    # the size of the matrix 
    size = 10
    grid = np.random.binomial(n,p, size=(size,size))
    first_row = grid[0]
    first_row[first_row == 1] = 0
    grid[0] = first_row
    for i in range(1,size):
        grid[i,size-1] = 1
    # create environment
    # from 0 to 899: map 
    # from 900 to 901: player position
    # from 902 to 903: start position
    # from 904 to 905: end position
    # 906: turn
    # 907: turn tối ưu
    env = np.full(size*9*size+8,0)

    # create the map 
    map = np.full((size*3, size*3),1)
    i = 0
    j = 0
    while i < size:
        w = i*3 + 1
        while j < size:
            k = j*3 + 1
            toss = grid[i,j]
            map[w,k] = 0 # ' '
            if toss == 0 and k+2 < size*3:
                map[w,k+1] = 0 # ' '
                map[w,k+2] = 0 #' '
            if toss == 1 and w-2 >=0:
                map[w-1,k] = 0 #' '
                map[w-2,k] = 0 #' '
            j = j + 1
        i = i + 1
        j = 0

    # create end and start point
    rows, cols = (np.where(map == 0))


    # start point
    start = random.randint(0, len(rows)-1)

    
    # end point 
    end = random.randint(0, len(rows)-1)
    if end == start :
        end = random.randint(0, len(rows)-1)

    
    map = map.reshape(1,size*9*size)[0]
    env[np.where(map== 1)[0]] = 1
    env[np.where(map== 0)[0]] = 0

    # current_agent_posistion_player
    env[size*9*size+0*2]= rows[start]
    env[size*9*size+0*2+1]= cols[start]

    # start position
    env[size*9*size+2] = rows[start]
    env[size*9*size+3] = cols[start]
    print('điểm bắt đầu', env[size*9*size+2],env[size*9*size+3])
    
    # end position
    env[size*9*size+4] = rows[end]
    env[size*9*size+5] = cols[end]
    print('điểm kết thúc', env[size*9*size+4],env[size*9*size+5])

    # Turn
    env[size*9*size+6] = 0
    #Turn tối ưu 
    env[size*9*size+7] = 100

    return env

@njit 
def getAgentState(e_state):
    size = 10
    p_state = np.full(10,0) 

    # The position of the agent
    p_state[0] = e_state[size*9*size+0*2] 
    p_state[1] = e_state[size*9*size+0*2+1]

    # The position of the end
    p_state[2] = e_state[size*9*size+4]
    p_state[3] = e_state[size*9*size+5] 
    # The posistion around the player

    # up
    if p_state[0] - 1 >= 0:
        p_state[4] = e_state[(p_state[0]-1)*size*3+p_state[1]]
    else:
        p_state[4] = 1

    # down
    if p_state[0] + 1 < size*3:
        p_state[5] = e_state[(p_state[0]+1)*size*3+p_state[1]]
    else:
        p_state[5] = 1

    # left
    if p_state[1] - 1 >= 0:
        p_state[6] = e_state[p_state[0]*size*3+p_state[1]-1]
    else:
        p_state[6] = 1

    # right
    if p_state[1] + 1 < size*3:
        p_state[7] = e_state[p_state[0]*size*3+p_state[1]+1]
    else:
        p_state[7] = 1
    #turn 
    p_state[8]= e_state[size*9*size+6]
    #turn tối ưu
    p_state[9]= e_state[size*9*size+7]
    return p_state


@njit
def stepEnv(action,env):
    size = 10
    pIdx = 0
    # current position
    cur_x = env[size*9*size+pIdx*2]
    cur_y = env[size*9*size+pIdx*2+1]

    # go straight
    if action == 1:
        if cur_x - 1 >= 0 and env[(cur_x-1)*size*3+cur_y] == 0:
            env[size*9*size+pIdx*2] = cur_x-1
            env[size*9*size+pIdx*2+1] = cur_y 

    # go down
    if action == 2:
        if cur_x + 1 < size*3 and env[(1+cur_x)*size*3+cur_y] == 0:
            env[size*9*size+pIdx*2] = cur_x+1
            env[size*9*size+pIdx*2+1] = cur_y 
    # go left
    if action == 0:
        if cur_y - 1 >= 0 and env[(cur_x)*size*3+cur_y-1] == 0:
            env[size*9*size+pIdx*2] = cur_x 
            env[size*9*size+pIdx*2+1] = cur_y-1

    # go right
    if action == 3:
        if cur_y + 1 < size*3 and env[(cur_x)*size*3+cur_y+1] == 0:
            env[size*9*size+pIdx*2] = cur_x 
            env[size*9*size+pIdx*2+1] = cur_y+1

    env[size*9*size+6]+=1  # Turn

@njit 
def getValidActions(p_state):
    validActions = np.full(4, 0)
    if p_state[4] != 1:#go up
        validActions[1] = 1
    if p_state[5] != 1: #go down
        validActions[2] = 1

    if p_state[6] != 1:# left
        validActions[0] = 1
    if p_state[7] != 1:# right
        validActions[3] = 1
    return validActions

# Visualize bàn chơi

In [ ]:
def player_random(p_state, temp_file, per_file2):
    arr_action = getValidActions(p_state)
    # print(p_state)
    arr_action = np.where(arr_action == 1)[0]
    act_idx = np.random.randint(0, len(arr_action))
    print(arr_action)
    return arr_action[act_idx], temp_file, per_file2


from os import environ
import numpy as np
from numba.typed import List

def run(listAgent, perData):
    size = 10
    env = initEnv()
    tempData = []
    dataOnePlayer = List()
    dataOnePlayer.append(np.array([[0.]]))
    tempData.append(dataOnePlayer)

    winner = -1

    

    
    while env[size*9*size+6]<env[size*9*size+7]:
        pIdx = 0
        output = env[0:size*9*size].reshape(size*3,size*3)
        output_grid1 = np.empty([size*3, size*3],dtype=str)
        output_grid1[:] = '#'
        for i in range(len(output)):
            for j in range(len(output)):
                if output[i,j] == 0:
                    output_grid1[i,j] = ' '
                if i == env[size*9*size] and j == env[size*9*size+1]:
                    output_grid1[i,j] = 'P1'
                if i == env[size*9*size+4] and j == env[size*9*size+5]:
                    output_grid1[i,j] = 'E'
        for elm in output_grid1:
            print(" ".join(elm))
        print(f'Turn {env[size*9*size+6]+1}',end=" ")
        action, tempData[pIdx], perData = listAgent[pIdx](getAgentState(env), tempData[pIdx], perData)
        if action == 0:  #go left   
            print(f'Player: {pIdx+1} đi trái')
        if action == 1 : # go straight :
            print(f'Player: {pIdx+1}  đi thẳng')
        if action == 2 : # go back :
            print(f'Player: {pIdx+1}  đi lùi')
        if action == 3 : # go right :
            print(f'Player: {pIdx+1}  đi phải')


        stepEnv(action, env)
        winner = checkEnded(env)
        
        if winner != -1:
            i = 0 
            print(f'P{i+1}:',end=" ")
            print('X',env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
            print(' ')
            print('số turn',env[size*9*size+6]+1)
            break
    
    action, tempData[pIdx], perData = listAgent[pIdx](getAgentState(env), tempData[pIdx], perData)
    
    return winner, perData


def main(listAgent, times, perData, printMode=False, k=100):
    if len(listAgent) != 1:
        raise Exception('Hệ thống chỉ cho phép có đúng 1 người chơi!!!')
    
    numWin = np.full(2, 0)
    pIdOrder = np.arange(1)
    for _ in range(times):
        if printMode and _ != 0 and _ % k == 0:
            print(_, numWin)

        winner, perData = run(listAgent, perData)

        if winner == -1:
            numWin[1] += 1
        else:
            numWin[0] += 1

    if printMode:
        print(_+1, numWin)

    return numWin, perData
list_player = [player_random]
per = []
count,per1 = main(list_player,1,per)

# Numba run 

In [8]:
from numba.typed import List

@njit
def numbaRun(p0, perData, pIdOrder):
    env= initEnv()
    tempData = []
    dataOnePlayer = List()
    dataOnePlayer.append(np.array([[0.]]))
    tempData.append(dataOnePlayer)
    size = 10
    winner = -1
    while env[size*9*size+6]< env[size*9*size+7] :
        pIdx = 0
        action, tempData[pIdx], perData = p0(getAgentState(env), tempData[pIdx], perData)
        stepEnv(action, env)
        winner = checkEnded(env)
        if winner != -1:
            break
    pIdx = 0
    action, tempData[pIdx], perData = p0(getAgentState(env), tempData[pIdx], perData)
            
    return winner, perData

@njit
def numbaMain(p0, times, perData, printMode=False, k=100):
    numWin = np.full(2, 0)
    pIdOrder = np.arange(1)
    for _ in range(times):
        if printMode and _ != 0 and _ % k == 0:
            print(_, numWin)

        np.random.shuffle(pIdOrder)
        winner, perData = numbaRun(p0, perData, pIdOrder)

        if winner == -1:
            numWin[1] += 1
        else:
            numWin[0] += 1
    
    if printMode:
        print(_+1, numWin)

    return numWin, perData
@njit
def numbaRandomBot(state, tempData, perData):
    validActions = getValidActions(state)
    validActions = np.where(validActions==1)[0]
    idx = np.random.randint(0, len(validActions))
    return validActions[idx], tempData, perData

perx = [np.array([np.random.rand(getActionSize()) for _ in range(100)]),np.zeros((1,100)),np.zeros((1,100)),np.zeros((1,100))]
count,per1 = numbaMain(numbaRandomBot,1000000,perx)
print(count)

/var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_4448/2625995191.py:37: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'perData' of function 'numbaRun'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_4448/2625995191.py", line 3:
<source missing, REPL/exec in use?>

  winner, perData = numbaRun(p0, perData, pIdOrder)
/var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_4448/2625995191.py:37: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'perData' of function 'numbaRandomBot'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

Fi

điểm bắt đầu 1 15
điểm kết thúc 1 13
số turn 3175
điểm bắt đầu 4 10
điểm kết thúc 13 8
số turn 31276
điểm bắt đầu 19 23
điểm kết thúc 22 25
số turn 1886
điểm bắt đầu 16 21
điểm kết thúc 1 17
số turn 402
điểm bắt đầu 10 3
điểm kết thúc 10 4
số turn 2
điểm bắt đầu 7 10
điểm kết thúc 25 19
số turn 11128
điểm bắt đầu 23 4
điểm kết thúc 4 16
số turn 2932
điểm bắt đầu 13 17
điểm kết thúc 8 7
số turn 5120
điểm bắt đầu 22 12
điểm kết thúc 7 21
số turn 19545
điểm bắt đầu 3 22
điểm kết thúc 4 27
số turn 85
điểm bắt đầu 7 10
điểm kết thúc 19 4
số turn 32471
điểm bắt đầu 5 28
điểm kết thúc 3 25
số turn 3524
điểm bắt đầu 19 4
điểm kết thúc 28 2
số turn 26058
điểm bắt đầu 13 1
điểm kết thúc 19 10
số turn 23982
điểm bắt đầu 28 1
điểm kết thúc 19 7
số turn 338
điểm bắt đầu 6 25
điểm kết thúc 25 28
số turn 7013
điểm bắt đầu 22 4
điểm kết thúc 1 28
số turn 13040
điểm bắt đầu 16 14
điểm kết thúc 4 21
số turn 21954
điểm bắt đầu 5 25
điểm kết thúc 21 28
số turn 342
điểm bắt đầu 19 7
điểm kết thúc 25 16
số 